## Análise pós-clusterização
***

In [156]:
import pandas as pd
import numpy as np

In [249]:
path = '/Users/felipebarreto/Desktop/TCC/Bases geradas/'
df1 = pd.read_csv('/Users/felipebarreto/Desktop/dados_acidentes_tratados_completo.csv', sep = ',')   ## dados originais tratados
df2 = pd.read_csv(path + 'centroides_padronizados.csv', sep = ';')   ## centróides de cada cluster     //
df3 = pd.read_csv(path + 'variaveis_quanti_agg.csv', sep = ';')      ## igual ao df4, sem padronização // 
df4 = pd.read_csv(path + 'variaveis_quanti_pad_agg.csv', sep = ';')  ## igual ao df3, com padronização // 
df5 = pd.read_csv(path + 'medias_var_cluster.csv', sep = ';', decimal= '.')        ## médias de cada cluster         // 
df6 = pd.read_csv(path + 'clusters_7.csv', sep = ';') # var quantitativas completas, nem todas foram utilizadas para a clusterização

In [245]:
df1.groupby('tracado_via_tipo_1')['tracado_via_tipo_1'].count().keys()

Index([False, True], dtype='object', name='tracado_via_tipo_1')

In [236]:
pd.DataFrame(df1.tracado_via_tipo_1.value_counts(normalize=True)).reset_index()

,index,tracado_via_tipo_1
0,True,0.707132
1,False,0.292868


In [254]:
df6

,municipio,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,freq,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane,cluster
0,curitiba,53273,581,10382,2872,36629,2808,13254,46614,23714,0.834781,0.155604,0.375221,0.438306,0.106477,1
1,sao jose,44027,191,8828,1928,31326,1752,10756,40653,20102,0.784648,0.205552,0.369018,0.414785,0.043379,1
2,guarulhos,40350,430,8282,1040,27409,3188,9322,36372,18109,0.942239,0.052018,0.410293,0.422055,0.055221,1
3,betim,39527,329,6840,1718,29551,1085,8558,35159,17601,0.784103,0.208625,0.361911,0.489234,0.048804,1
4,duque de caxias,37859,407,6388,1273,28066,1723,7661,32465,17531,0.842393,0.153785,0.380811,0.430152,0.195710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,viamao,2,0,0,0,2,0,0,2,1,1.000000,0.000000,0.000000,0.000000,0.000000,3
2259,vila pavao,1,0,0,0,1,0,0,1,1,1.000000,0.000000,0.000000,0.000000,1.000000,2
2260,virginopolis,2,0,1,0,1,0,1,1,1,0.000000,1.000000,0.000000,1.000000,1.000000,4
2261,vitorino freire,2,0,0,0,2,0,0,2,1,1.000000,0.000000,0.000000,0.000000,1.000000,2


In [288]:
def _status_qtd(var):
    a = pd.merge(df1, df6[['municipio','cluster']], how='left')\
          .groupby(['cluster',var])[['id']]\
          .count()\
          .reset_index()\
          .rename(columns={var:'status','id':var})\
          .set_index(['cluster','status'])
    return a

In [293]:
_status_qtd('tracado_via_tipo_1')\
    .join(_status_qtd('is_weekend'))\
    .join(_status_qtd('is_night'))\
    .join(_status_qtd('is_single_lane'))

tracado_via_tipo_1  is_weekend  is_night  is_single_lane
cluster status                                                          
1       False                62988      175841    199847          247456
        True                249718      136865    112859           65250
2       False                80012      175564    200865           20302
        True                263650      168098    142797          323360
3       False                65031      114960    131837          185778
        True                153379      103450     86573           32632
4       False                86222       78425     95846            7071
        True                 66154       73951     56530          145305
5       False               214713      379447    440833          368479
        True                496002      331268    269882          342236

In [285]:
pd.join(_status_qtd('is_night'),_status_qtd('is_weekend'))

AttributeError: module 'pandas' has no attribute 'join'

In [283]:
_status_qtd('is_night')

,cluster,status,is_night
0,1,False,199847
1,1,True,112859
2,2,False,200865
3,2,True,142797
4,3,False,131837
5,3,True,86573
6,4,False,95846
7,4,True,56530
8,5,False,440833
9,5,True,269882


In [282]:
pd.merge(
    _status_qtd('tracado_via_tipo_1'),_status_qtd('is_weekend'),_status_qtd('is_night')
)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [280]:
_status_qtd('tracado_via_tipo_1')\
    .pd.merge(_status_qtd('is_weekend'), how=['cluster','status'])

AttributeError: 'DataFrame' object has no attribute 'pd'

In [273]:
pd.merge(df1, df6[['municipio','cluster']], how='left')\
    .groupby(['cluster','tracado_via_tipo_1'])[['id']]\
    .count()\
    .reset_index()\
    .rename(columns={'tracado_via_tipo_1':'status','id':'tracado_via_tipo_1'})

,cluster,status,tracado_via_tipo_1
0,1,False,62988
1,1,True,249718
2,2,False,80012
3,2,True,263650
4,3,False,65031
5,3,True,153379
6,4,False,86222
7,4,True,66154
8,5,False,214713
9,5,True,496002


In [252]:
df6.groupby(['cluster','share_tracado_via_tipo_1'])[['cluster']].count()

cluster
cluster share_tracado_via_tipo_1         
1       0.632138                        1
        0.669109                        1
        0.680138                        1
        0.694181                        1
        0.706108                        1
...                                   ...
5       0.960695                        1
        0.963798                        1
        0.967316                        1
        0.971684                        1
        0.972609                        1

[1602 rows x 1 columns]

In [158]:
display(
    df2.style.background_gradient(cmap='Greens').set_precision(2),
    df5.style.background_gradient(cmap='Reds').set_precision(2)
    )

,Variavel,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5
0,mortos,5.11,-0.22,-0.16,-0.32,1.84
1,feridos_leves,6.54,-0.28,-0.02,-0.26,1.70
2,feridos_graves,6.46,-0.26,-0.12,-0.27,1.74
3,ilesos,7.03,-0.26,-0.01,-0.26,1.51
4,freq,6.90,-0.28,0.01,-0.26,1.62
5,share_tracado_via_tipo_1,0.42,0.50,0.15,-1.37,0.02
6,share_is_night,-0.25,0.10,-0.14,-0.11,-0.13
7,share_is_weekend,-0.23,0.04,-0.23,0.06,-0.05
8,share_is_single_lane,-1.83,0.45,-2.11,0.41,-0.78


,cluster,mortos,feridos_leves,feridos_graves,ilesos,freq,share_tracado_via_tipo_1,share_is_night,share_is_weekend,share_is_single_lane
0,Cluster 1,330.33,5434.96,1752.12,20100.00,13029.42,0.80,0.36,0.44,0.24
1,Cluster 2,27.04,154.29,66.81,352.11,271.88,0.82,0.43,0.49,0.96
2,Cluster 3,30.57,351.57,101.76,1023.44,791.34,0.73,0.38,0.44,0.15
3,Cluster 4,21.64,170.06,63.53,358.45,302.33,0.36,0.39,0.49,0.95
4,Cluster 5,144.31,1689.63,568.29,5151.79,3644.69,0.70,0.38,0.47,0.57


### Análise

In [166]:
df = df3.drop(['pessoas','ignorados','feridos','veiculos'], axis=1)[[
    'municipio','cluster','freq','mortos','feridos_graves','feridos_leves','ilesos','share_tracado_via_tipo_1',
    'share_tracado_via_tipo_2','share_is_night','share_is_weekend','share_is_single_lane'
]].drop_duplicates(subset = ['municipio'])


## Separação dos clusters (c1,c2,c3,c4,c5,c6,c7)
n = len(df.cluster.drop_duplicates())
for i in range(1,n + 1):
    globals()['c' + str(i)] = df[df.cluster == i].drop(['cluster'], axis=1)

df.head(5).style.background_gradient(cmap='Blues').set_precision(2)

,municipio,cluster,freq,mortos,feridos_graves,feridos_leves,ilesos,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
0,curitiba,1,23714,581,2872,10382,36629,0.83,0.16,0.38,0.44,0.11
1,sao jose,1,20102,191,1928,8828,31326,0.78,0.21,0.37,0.41,0.04
2,guarulhos,1,18109,430,1040,8282,27409,0.94,0.05,0.41,0.42,0.06
3,betim,1,17601,329,1718,6840,29551,0.78,0.21,0.36,0.49,0.05
4,duque de caxias,1,17531,407,1273,6388,28066,0.84,0.15,0.38,0.43,0.20


In [204]:
c1.mortos.sum()

7928

In [205]:
c1.mortos.mean()

330.3333333333333

In [160]:
def _loc(df, cluster, col):
    return(round(df[df.cluster == 'Cluster '+str(cluster)][col].iloc[0],2))

In [171]:
round(df.mortos.mean(),2)

39.59

In [193]:
globals()['mortos_c' + str(1)] = 2

In [219]:
## Tamanho dos clusters // Quantidade de municípios
print(
    f""">> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: {len(df)}
    - Mortos = {df.mortos.sum()} | Média = {round(df.mortos.mean(),2)}
    - Feridos graves = {df.feridos_graves.sum()} | Média = {round(df.feridos_graves.mean(),2)}
    - Feridos leves = {df.feridos_leves.sum()} | Média = {round(df.feridos_leves.mean(),2)}
    - Ilesos = {df.ilesos.sum()} | Média = {round(df.ilesos.mean(),2)}
    """
)

for i in range(1,n+1):

    print(f"""Cluster {i}:
        - {len(globals()['c' + str(i)])} municípios --> {round(100*df[df.cluster == i].shape[0] / df.shape[0],2)}% do total
        - Mortos = {globals()['c' + str(i)].mortos.sum()},  {round(100*globals()['c' + str(i)].mortos.sum() / df.mortos.sum(),2)}% do total | Média = {_loc(df5, i, col='mortos')}, {round(100*_loc(df5, i, col='mortos') / df.mortos.mean(),2)}% da média geral
        - Feridos graves = {globals()['c' + str(i)].feridos_graves.sum()},  {round(100*globals()['c' + str(i)].feridos_graves.sum() / df.feridos_graves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_graves')}, {round(100*_loc(df5, i, col='feridos_graves') / df.feridos_graves.mean(),2)}% da média geral
        - Feridos Leves = {globals()['c' + str(i)].feridos_leves.sum()},  {round(100*globals()['c' + str(i)].feridos_leves.sum() / df.feridos_leves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_leves')}, {round(100*_loc(df5, i, col='feridos_leves') / df.feridos_leves.mean(),2)}% da média geral
        - Ilesos = {globals()['c' + str(i)].ilesos.sum()},  {round(100*globals()['c' + str(i)].ilesos.sum() / df.ilesos.sum(),2)}% do total | Média = {_loc(df5, i, col='ilesos')}, {round(100*_loc(df5, i, col='ilesos') / df.ilesos.mean(),2)}% da média geral
        - Média do share de traçado via tipo 1 = {_loc(df5, i, col='share_tracado_via_tipo_1')}
        - Média do share de é noite = {_loc(df5, i, col='share_is_night')}
        - Média do share de é final de semana = {_loc(df5, i, col='share_is_weekend')}
        - Média do share de é pista simples = {_loc(df5, i, col='share_is_single_lane')}
    """)

>> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: 2263
    - Mortos = 89587 | Média = 39.59
    - Feridos graves = 297414 | Média = 131.42
    - Feridos leves = 837677 | Média = 370.16
    - Ilesos = 2395201 | Média = 1058.42
    
Cluster 1:
        - 24 municípios --> 1.06% do total
        - Mortos = 7928,  8.85% do total | Média = 330.33, 834.43% da média geral
        - Feridos graves = 42051,  14.14% do total | Média = 1752.12, 1333.17% da média geral
        - Feridos Leves = 130439,  15.57% do total | Média = 5434.96, 1468.26% da média geral
        - Ilesos = 482400,  20.14% do total | Média = 20100.0, 1899.06% da média geral
        - Média do share de traçado via tipo 1 = 0.8
        - Média do share de é noite = 0.36
        - Média do share de é final de semana = 0.44
        - Média do share de é pista simples = 0.24
    
Cluster 2:
        - 1264 municípios --> 55.86% do total
        - Mortos = 34174,  38.15% do total | Média = 

In [12]:
#df5 = df5.set_index('cluster')

df5.style.background_gradient(cmap='Greens')\
    #.background_gradient(cmap='coolwarm', axis=None, vmin=0, vmax=1)
    #.highlight_null(null_color='#f1f1f1')  # Color NaNs grey
    .set_precision(2)

,cluster,mortos,feridos_leves,feridos_graves,ilesos,freq,share_tracado_via_tipo_1,share_is_night,share_is_weekend,share_is_single_lane
0,Cluster 1,330.33,5434.96,1752.12,20100.00,13029.42,0.80,0.36,0.44,0.24
1,Cluster 2,27.04,154.29,66.81,352.11,271.88,0.82,0.43,0.49,0.96
2,Cluster 3,30.57,351.57,101.76,1023.44,791.34,0.73,0.38,0.44,0.15
3,Cluster 4,21.64,170.06,63.53,358.45,302.33,0.36,0.39,0.49,0.95
4,Cluster 5,144.31,1689.63,568.29,5151.79,3644.69,0.70,0.38,0.47,0.57


#### Observações:

- Total de Municípios analisados: 2263
- Municípios por região:
    - N ---> 
    - NE --> 
    - CO --> 
    - S ---> 
    - SE -->  
- Municípios por UF:
    - sdc

## Cluster 1

In [105]:
df5[df5.cluster == 'Cluster 5'].feridos_leves.iloc[0]

1689.63076923077

In [101]:
c1.shape

(493, 10)

In [ ]:
c1.groupby('uf')[['freq']].sum().sort_values(by = ['freq'], ascending = False)

,freq
uf,
mg,33232
sc,28153
rs,21568
ba,13154
pr,12833
es,6291
al,4140
rj,1772
go,1314


In [ ]:
c1.head(50)

,uf,municipio,freq,mortos,feridos_graves,feridos_leves,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
312,sc,aguas mornas,1520,77,331,906,0.294737,0.681579,0.325658,0.506579,0.996053
331,mg,bela vista de minas,1408,62,366,930,0.182528,0.799006,0.287642,0.477273,0.990767
346,sc,sao cristovao do sul,1372,71,209,682,0.420554,0.561953,0.389942,0.471574,0.990525
350,mg,campos altos,1357,83,219,796,0.275608,0.708180,0.280766,0.464996,0.973471
354,sc,rancho queimado,1335,66,260,742,0.223221,0.759551,0.284644,0.528090,0.977528
366,es,rio novo do sul,1255,81,243,579,0.431076,0.566534,0.333068,0.451793,0.985657
372,sc,xaxim,1224,63,200,681,0.403595,0.582516,0.406046,0.489379,0.992647
377,sc,alfredo wagner,1201,70,251,816,0.214821,0.771857,0.383014,0.512073,0.993339
379,pr,pato branco,1196,32,190,666,0.529264,0.448161,0.401338,0.433110,0.921405
380,sc,ponte alta,1194,60,193,586,0.440536,0.536013,0.403685,0.474874,0.981575
